In [ ]:
%pip install --quiet --upgrade diffusers transformers scipy mediapy ipyfilechooser

In [ ]:
from diffusers import PNDMScheduler, DDIMScheduler, LMSDiscreteScheduler
import mediapy as media
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline
import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from ipyfilechooser import FileChooser
import PIL
from PIL import Image
plt.ioff();

In [ ]:
def init_model():
  global auth_token
  global pipe, scheduler

  scheduler = PNDMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", skip_prk_steps=True)
  # scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear")
  # scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear")

  model_id = "CompVis/stable-diffusion-v1-4"
  device = "cuda"
  remove_safety = True

  pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16, revision="fp16", use_auth_token=auth_token)
  if remove_safety:
    pipe.safety_checker = lambda images, clip_input: (images, False)
  pipe = pipe.to(device)


def generate_image():
  global prompt
  global pipe
  global guidance_scale
  global num_inference_steps
  global images
  global init_image
  
  try: init_image
  except: init_image=None

  num_images = 1

  prompts = [ prompt ] * num_images
  with autocast("cuda"):
      images = pipe(prompts, guidance_scale=guidance_scale, init_image=init_image ,num_inference_steps=num_inference_steps)["sample"]  
      
  media.show_images(images)

def reset_init_image():
  global init_image
  init_image = None


In [ ]:
def simple_ui():
    button_clear_output = widgets.Button(description='表示クリア')
    button_init_model = widgets.Button(description='モデル初期化')
    button_generate_image = widgets.Button(description='生成')
    button_reset_init_image = widgets.Button(description='init_image消去')
    filechooser = FileChooser('./')
    button_load_as_init_image = widgets.Button(description='init_image読込')
    
    button_input_field = widgets.Button(description='変数反映')

    text_auth_token = widgets.Text(value='',description='auth_token')
    text_prompt = widgets.Text(value='',description='prompt',layout= widgets.Layout(width='100%'))
    float_guidance_scale = widgets.FloatText(value=7.5, description='guidance_scale')
    int_num_inference_steps = widgets.IntText(value=50, description='num_inference_steps')

    output = widgets.Output()
    def wrapped_func_factory(func):
        def new_func(ui_element):
            with output:
                print(f"exec func {func.__name__}")
                func()
                print(f"complete {func.__name__}")
        return new_func
    button_clear_output.on_click(lambda button: output.clear_output(wait=False))
    button_init_model.on_click(wrapped_func_factory(init_model))
    button_generate_image.on_click(wrapped_func_factory(generate_image))
    button_reset_init_image.on_click(wrapped_func_factory(reset_init_image))

    def load_as_init_image():
        global init_image
        path = filechooser.selected
        image = Image.open(path).convert("RGB")
        w, h = image.size
        print(f"loaded input image of size ({w}, {h}) from {path}")
        w, h = map(lambda x: x - x % 32, (w, h))  # resize to integer multiple of 32
        image = image.resize((w, h), resample=PIL.Image.LANCZOS)
        image = np.array(image).astype(np.float32) / 255.0
        image = image[None].transpose(0, 3, 1, 2)
        image = torch.from_numpy(image)
        init_image = 2.*image - 1.
    button_load_as_init_image.on_click(wrapped_func_factory(load_as_init_image))
    
    def load_input_field():
        global auth_token
        global prompt
        global guidance_scale
        global num_inference_steps
        auth_token = text_auth_token.value
        prompt = text_prompt.value
        guidance_scale = float_guidance_scale.value
        num_inference_steps = int_num_inference_steps.value
    button_input_field.on_click(wrapped_func_factory(load_input_field))

    display(
        widgets.HBox([button_clear_output, button_init_model, button_generate_image,button_reset_init_image]),
        widgets.HBox([filechooser,button_load_as_init_image]),
        widgets.Accordion(children=[
            widgets.VBox([
                widgets.HBox([text_auth_token, float_guidance_scale, int_num_inference_steps ,button_input_field]),
                widgets.HBox([text_prompt]),
            ])
        ],  _titles={0:'各種設定'}, selected_index=None),
        output,)

In [ ]:
simple_ui()